In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
# Load the dataset
df = pd.read_csv('house_price.csv')

# View the first few rows
print("First 5 rows of the dataset:")
print(df.head())

# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())


First 5 rows of the dataset:
   size  bedroom   price
0  2104        3  399900
1  1600        3  329900
2  2400        3  369000
3  1416        2  232000
4  3000        4  539900

Missing values in each column:
size       0
bedroom    0
price      0
dtype: int64


In [10]:
# ****** Linear Regression ******
# Prepare the data
X = df[['size', 'bedroom']]   # Features
y = df['price']               # Target

# Train the model
lr_model = LinearRegression()
lr_model.fit(X, y)

# Predict using the model
y_pred = lr_model.predict(X)

# Evaluate the model
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y - y_pred) / y)) * 100

# Output results
print("==== Linear Regression ====")
print("Coefficients:", lr_model.coef_)
print("Intercept:", lr_model.intercept_)
print(f"MAE: {mae_lr:.2f}")
print(f"MSE: {mse_lr:.2f}")
print(f"RMSE: {rmse_lr:.2f}")
print(f"MAPE: {mape_lr:.2f}%")

# ******  SDG Regressor  *****
# Standardize features (important for SGD)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the SGDRegressor
sgd_model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
sgd_model.fit(X_scaled, y)

# Predict
y_pred_sgd = sgd_model.predict(X_scaled)

# Evaluate
mae_sgd = mean_absolute_error(y, y_pred_sgd)
mse_sgd = mean_squared_error(y, y_pred_sgd)
rmse_sgd = np.sqrt(mse_sgd)
mape_sgd = np.mean(np.abs((y - y_pred_sgd) / y)) * 100

# Output results
print("\n[SGDRegressor]")
print("Coefficients:", sgd_model.coef_)
print("Intercept:", sgd_model.intercept_)
print(f"MAE: {mae_sgd:.2f}")
print(f"MSE: {mse_sgd:.2f}")
print(f"RMSE: {rmse_sgd:.2f}")
print(f"MAPE: {mape_sgd:.2f}%")


==== Linear Regression ====
Coefficients: [  139.21067402 -8738.01911233]
Intercept: 89597.90954279754


NameError: name 'mae_lr' is not defined

In [8]:
# Standardize features (important for SGD)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the SGDRegressor
sgd_model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
sgd_model.fit(X_scaled, y)

# Predict
y_pred_sgd = sgd_model.predict(X_scaled)

# Evaluate
mae_sgd = mean_absolute_error(y, y_pred_sgd)
mse_sgd = mean_squared_error(y, y_pred_sgd)
rmse_sgd = np.sqrt(mse_sgd)
mape_sgd = np.mean(np.abs((y - y_pred_sgd) / y)) * 100

# Output results
print("\n[SGDRegressor]")
print("Coefficients:", sgd_model.coef_)
print("Intercept:", sgd_model.intercept_)
print(f"MAE: {mae_sgd:.2f}")
print(f"MSE: {mse_sgd:.2f}")
print(f"RMSE: {rmse_sgd:.2f}")
print(f"MAPE: {mape_sgd:.2f}%")



[SGDRegressor]
Coefficients: [108873.92893799  -5981.12043584]
Intercept: [340375.84521395]
MAE: 51563.18
MSE: 4086863630.42
RMSE: 63928.58
MAPE: 15.62%
